# Master Data Science for Business - Data Science Consulting - Session 2 

# Notebook 2: 

# Web Scraping with Scrapy: Getting reviews from TripAdvisor

To Do (note for Cap): <br>
-Enlever des parties du code que les élèves doivent compléter par eux même 

## 1. Importing packages

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.spiders import CrawlSpider, Rule
from scrapy.selector import Selector
import sys
from scrapy.http import Request
from scrapy.linkextractors import LinkExtractor
import json
import logging
import pandas as pd

## 2. Some class and functions

In [2]:
# -*- coding: utf-8 -*-

# Define here the models for your scraped items
#
# See documentation in:
# https://doc.scrapy.org/en/latest/topics/items.html

class HotelreviewsItem(scrapy.Item):
    # define the fields for your item here like:
    rating = scrapy.Field()
    review = scrapy.Field()
    title = scrapy.Field()
    trip_date = scrapy.Field()
    trip_type = scrapy.Field()
    published_date = scrapy.Field()
    hotel_type = scrapy.Field()
    hotel_name = scrapy.Field()
    price_range = scrapy.Field()
    reviewer_id = scrapy.Field()
    review_language = scrapy.Field()

In [3]:
def user_info_splitter(raw_user_info):
    """

    :param raw_user_info:
    :return:
    """

    user_info = {}

    splited_info = raw_user_info.split()
    for element in splited_info:
        converted_element = get_convertible_elements_as_dic(element)
        if converted_element:
            user_info[converted_element[0]] = converted_element[1]

    return user_info

## 2. Creating the JSon pipeline 

In [4]:
#JSon pipeline, you can rename the "trust.jl" to the name of your choice
class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('tripadvisor.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

## 3. Spider

When you go on a TripAdvisor page, you will have 5 reviews per page. Reviews are not fully displayed on the page, so you have to open them (i.e follow the link of the review to tell Scrapy to scrape this page) to scrape them. <br>
This means we will use 2 parsing functions: <br>
-The first one will go on the page of the parc, and get the links of the reviews <br>
-The second one will go on each page of each reviews and scrape them using the parse_item() method. <br>

<b>To Do</b>: Complete the code with XPath to get the proper item to scrape. Once you are done, you can "Restart and run all cells" to see if everything is working correctly. 

In [11]:
class MySpider(CrawlSpider):
    name = 'BasicSpider'
    domain_url = "https://www.tripadvisor.com"
    # allowed_domains = ["https://www.tripadvisor.com"]

    start_urls = [
        "https://www.tripadvisor.fr/ShowUserReviews-g1573379-d1573383-r629218790-Center_Parcs_Les_Trois_Forets-Hattigny_Moselle_Grand_Est.html",
    "https://www.tripadvisor.fr/ShowUserReviews-g1573379-d1573383-r645720538-Center_Parcs_Les_Trois_Forets-Hattigny_Moselle_Grand_Est.html"]
    
        #Custom settings to modify settings usually found in the settings.py file 
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'tripadvisor.json'                        # Used for pipeline 2
    }

    def parse(self, response):

        item = HotelreviewsItem()

        item["reviewer_id"] = next(iter(response.xpath(
            "//div[contains(@class,'prw_reviews_resp_sur_h_featured_review')]/div/div/div/div/div[contains(@class,'prw_reviews_user_links_hs')]/span/@data-memberid").extract()),
                                   None)
        item["review_language"] = next(iter(response.xpath(
            "//div[contains(@class,'prw_reviews_resp_sur_h_featured_review')]/div/div/div/div/div[contains(@class,'prw_reviews_user_links_hs')]/span/@data-language").extract()),
                                       None)

        review_url_on_page = response.xpath('//script[@type="application/ld+json"]/text()').extract()
        review = eval(review_url_on_page[0])

        item["review"] = review["reviewBody"].replace("\\n", "")
        item["title"] = review["name"]
        item["rating"] = review["reviewRating"]["ratingValue"]
        item["hotel_type"] = review["itemReviewed"]["@type"]
        item["hotel_name"] = review["itemReviewed"]["name"]
        item["price_range"] = review["itemReviewed"]["priceRange"]
        try:
            item["published_date"] = review["datePublished"]
        except KeyError:

            item["published_date"] = next(iter(response.xpath(
                f"//div[contains(@id,'review_{review_id}')]/div/div/span[@class='ratingDate']/@title""").extract()),
                                          None)

        item["trip_type"] = next(iter(response.xpath("//div[contains(@class,"
                                                     "'prw_reviews_resp_sur_h_featured_review')]/div/div/div/div/div"
                                                     "/div/div/div[contains(@class,'noRatings')]/text()").extract()),
                                 None)

        try:
            item["trip_date"] = next(iter(response.xpath("//div[contains(@class,"
                                                         "'prw_reviews_resp_sur_h_featured_review')]/div/div/div/div["
                                                         "contains(@class,'prw_reviews_stay_date_hsx')]/text()").extract(

            )), None)

        except:

            item["trip_date"] = next(iter(response.xpath(
                "//div[contains(@id,'review_538163624')]/div/div/div[@data-prwidget-name='reviews_stay_date_hsx']/text()").extract()),
                                     None)

        yield item


## 4. Crawling

In [12]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(MySpider)
process.start()

2019-01-21 19:32:02 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2019-01-21 19:32:02 [scrapy.utils.log] INFO: Versions: lxml 4.3.0.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.19.0, Twisted 17.5.0, Python 3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) - [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 18.0.0 (OpenSSL 1.1.1a  20 Nov 2018), cryptography 2.4.2, Platform Darwin-15.6.0-x86_64-i386-64bit
2019-01-21 19:32:02 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'tripadvisor.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


ReactorNotRestartable: 

## 5. Importing and reading data scraped

If you've succeeded, you should see here a dataframe with 2 entries corresponding to the 2 first reviews of the parc, and 11 columns for each item scraped. 

In [13]:
dfjson = pd.read_json('tripadvisor.json')
#Previewing DF
dfjson.head()

,hotel_name,hotel_type,price_range,published_date,rating,review,review_language,reviewer_id,title,trip_date,trip_type
0,Center Parcs - Les Trois Forêts,LodgingBusiness,82€ - 246€ (Selon les tarifs moyens d'une cha...,14 janvier 2019,5,c\'est le troisi\u00e8me center parc qu\'on vi...,fr,<div>Sightsee26280840290</div>,"logement tout equip\u00e9, luxury",janvier 2019,None
1,Center Parcs - Les Trois Forêts,LodgingBusiness,82€ - 246€ (Selon les tarifs moyens d'une cha...,28 octobre 2018,5,Seconde fois que nous venons au Center Parcs d...,fr,<div>Kathys25</div>,D\u00e9paysant!,septembre 2018,A voyagé en famille


In [8]:
dfjson.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 11 columns):
hotel_name         2 non-null object
hotel_type         2 non-null object
price_range        2 non-null object
published_date     2 non-null object
rating             2 non-null int64
review             2 non-null object
review_language    2 non-null object
reviewer_id        2 non-null object
title              2 non-null object
trip_date          2 non-null object
trip_type          1 non-null object
dtypes: int64(1), object(10)
memory usage: 256.0+ bytes
